This was to create an LDA model, but the results are bad and undesirable, which I think is the result of the source data. I'm going to move forward with analyzing the data based on Counts/CountVectorizers.

In [31]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
import json 
import logging
from pathlib import Path

from pjutils import create_fh_logger

In [32]:
# locations of json files + a place to store a log
src = Path.cwd().parent.parent / 'processing' / 'nro_declassified' / 'tokenized'
dst = src.parent / 'topics'
files = list(src.glob('*json'))
logs = Path.cwd() / 'logs'
logs.mkdir(exist_ok=True)
logger = create_fh_logger(logs / "topic_model.log")

In [33]:
documents = []
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
    words = [item['word'] for item in data]
    documents.append(preprocess_string((' ').join(words))) # some edge cases I still missed, letting gensim pick up the slack here

In [45]:
dictionary = Dictionary(documents)
dictionary.filter_extremes(no_below=0.1, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in documents]
lda_model = LdaModel(corpus=corpus, id2word=dictionary, per_word_topics=True,  num_topics=6, passes=5)

In [46]:
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

Topic: 0 
Words: 0.025*"camera" + 0.018*"frame" + 0.014*"imag" + 0.012*"pass" + 0.011*"control" + 0.009*"corona" + 0.008*"film" + 0.008*"handl" + 0.008*"qualiti" + 0.007*"target"

Topic: 1 
Words: 0.020*"corona" + 0.014*"program" + 0.008*"launch" + 0.008*"camera" + 0.007*"satellit" + 0.007*"air" + 0.006*"develop" + 0.006*"reconnaiss" + 0.006*"project" + 0.006*"success"

Topic: 2 
Words: 0.020*"film" + 0.015*"color" + 0.012*"camera" + 0.010*"us" + 0.010*"area" + 0.010*"filter" + 0.008*"resolut" + 0.008*"target" + 0.007*"test" + 0.007*"fig"

Topic: 3 
Words: 0.017*"camera" + 0.008*"tee" + 0.006*"film" + 0.006*"frame" + 0.005*"exposur" + 0.005*"data" + 0.005*"test" + 0.005*"perform" + 0.005*"flight" + 0.004*"rev"

Topic: 4 
Words: 0.011*"shall" + 0.011*"test" + 0.011*"vehicl" + 0.010*"command" + 0.010*"recoveri" + 0.009*"orbit" + 0.008*"control" + 0.007*"payload" + 0.006*"camera" + 0.006*"data"

Topic: 5 
Words: 0.012*"test" + 0.009*"data" + 0.008*"film" + 0.008*"orbit" + 0.007*"us" + 0.0

In [47]:
for document_num, doc in enumerate(corpus):
    doc_topics = lda_model.get_document_topics(doc)
    # Optionally, sort the topics by their contribution to this document
    doc_topics_sorted = sorted(doc_topics, key=lambda x: x[1], reverse=True)
    for topic_num, prop_topic in doc_topics_sorted:
        # Print the top topic and its contribution
        if topic_num == 4:
            print(f"Document {document_num} Top Topics: {doc_topics}")
            print(f'File: {files[document_num]}')
            print(f"Top topic: {topic_num}, Contribution: {prop_topic:.4f}")
            # Print top words for this topic
            word_contributions = lda_model.show_topic(topic_num)
            print(f"Top words for top topic: {word_contributions}")
            print("\n")

Document 0 Top Topics: [(1, 0.106431626), (2, 0.08089316), (4, 0.40099505), (5, 0.41152713)]
File: c:\Users\brasw\Desktop\School\Spring 24\GGS 590\project\nro_declassified\tokenized\1956-06-26MEMO PHYSICAL RECOVERY OF SATELLITE PAYLOADS A PRELIMINARY I_586.json
Top topic: 4, Contribution: 0.4010
Top words for top topic: [('shall', 0.011066876), ('test', 0.01066772), ('vehicl', 0.010578858), ('command', 0.0101726465), ('recoveri', 0.009998093), ('orbit', 0.008911957), ('control', 0.007944583), ('payload', 0.0068178764), ('camera', 0.00640777), ('data', 0.0063212784)]


Document 4 Top Topics: [(0, 0.029745892), (1, 0.2346921), (2, 0.09691832), (3, 0.019097326), (4, 0.2533436), (5, 0.36620274)]
File: c:\Users\brasw\Desktop\School\Spring 24\GGS 590\project\nro_declassified\tokenized\1957-11-12DOC RAND RESEARCH MEMORANDUM A FAMILY OF RECOVERABLE RECONNA_588.json
Top topic: 4, Contribution: 0.2533
Top words for top topic: [('shall', 0.011066876), ('test', 0.01066772), ('vehicl', 0.010578858)